**Inputs**
* NEX-GDDP precip

**Outputs**
* One ImageCollection per model, with 95th percentile or ARI for each pixel

In [1]:
import ee
ee.Authenticate()

Enter verification code: 4/1AX4XfWhj9jx_uXGcgrf1CFv3R421gHIGMGKWJa0gl7raT7lQsgi8ijN8m9g

Successfully saved authorization token.


In [2]:
ee.Initialize()

In [66]:
from datetime import date, timedelta
import sys

In [81]:
hist_data = ee.ImageCollection("NASA/NEX-GDDP").select('pr')
#PR_99 = ee.ImageCollection('users/tedwongwri/dataportal/percentiles/pr_99')
#TASMAX_99 = ee.ImageCollection('users/tedwongwri/dataportal/percentiles/tasmax_99'),
#TASMIN_01 = ee.ImageCollection('users/tedwongwri/dataportal/percentiles/tasmin_01'),
SHORT_GLOBE = ee.Geometry.Rectangle([-179.999, -60, 180, 72], 'EPSG:4326', False)
MODELS = ['ACCESS1-0', 'bcc-csm1-1', 'BNU-ESM', 'CanESM2', 'CCSM4', 'CESM1-BGC', 'CNRM-CM5', 'CSIRO-Mk3-6-0', 'GFDL-CM3', 'GFDL-ESM2G', 'GFDL-ESM2M', 'inmcm4', 'IPSL-CM5A-LR', 'IPSL-CM5A-MR', 'MIROC-ESM', 'MIROC-ESM-CHEM', 'MIROC5', 'MPI-ESM-LR', 'MPI-ESM-MR', 'MRI-CGCM3', 'NorESM1-M']
START_DATE = date(2000, 1, 1)
END_DATE = date(2000, 12, 31)

In [64]:
def ari95(start_date, end_date, model):
# antecedent rainfall index from https://agupubs.onlinelibrary.wiley.com/doi/full/10.1002/2017EF000715
# takes start, end dates as datetime.date and model name
# returns ee.ImageCollection
    model_collection = ee.ImageCollection(ee.List([]))
    modeldata = hist_data.filterMetadata('model', 'equals', model)
    precip_list = ee.List([])
    for t in range(6):
        tt = 6 - t
        tdate = start_date - timedelta(days=tt)
        modelscenario_data = modeldata.filterMetadata('scenario', 'equals', ['historical', 'rcp85'][(tdate.year > 2005) * 1])
        precip_list.add(modelscenario_data.filterDate(tdate.isoformat(), (tdate + timedelta(days=1)).isoformat()).first().multiply(86400))
    for datediff in range((end_date - start_date).days + 1):
        focal_date = start_date + timedelta(days = datediff)
        modelscenario_data = modeldata.filterMetadata('scenario', 'equals', ['historical', 'rcp85'][(focal_date.year > 2005) * 1])
        precip_list = precip_list.slice(0, 6)
        precip_list.add(modelscenario_data.filterDate(focal_date.isoformat(), (focal_date + timedelta(days=1)).isoformat()).first().multiply(86400))
        numerator = ee.Image.constant(0)
        denominator = ee.Image.constant(0)
        for t in range(7):
            tdate = focal_date - timedelta(days=t)
            precip = ee.Image(precip_list.get(-(t + 1)))
            w = 1.0 / ((t + 1)**2)
            numerator = numerator.add(precip.multiply(w))
            denominator = denominator.add(w)
        model_collection = model_collection.combine(ee.ImageCollection([numerator.divide(denominator)]))
    #model_collection = ee.ImageCollection(model_list)
    return model_collection.reduce(ee.Reducer.percentile([95]))

In [73]:
sys.setrecursionlimit(2000)

In [83]:
for model in MODELS:
    task = ee.batch.Export.image.toAsset(**{
      'image': ari95(START_DATE, END_DATE, model).set('model', model),
      'description': 'ari95_{}'.format(model),
      'assetId': 'users/tedwongwri/dataportal/landslide/ari95/{0}'.format(model),
      'region': SHORT_GLOBE,
      'crs': 'EPSG:4326',
      'dimensions': '43201x15841'
    })
    task.start()